In [1]:
from glob import glob
from pandas import read_csv, DataFrame
from framed import Gene, FBA, load_cbmodel, save_cbmodel
from framed.cobra.deletion import deleted_genes_to_reactions 
from framed.solvers import solver_instance
from framed.solvers.solver import Status, VarType
from collections import OrderedDict

### Get list with strain identifiers

In [2]:
def load_strain_ids():
    data = read_csv('../data/strain_list.txt', sep='\t', header=None)#, nrows=1)
    return OrderedDict(zip(data[0], data[1]))

### Get list of modified genes for a strain (mitochondrial genes excluded)

In [3]:
def get_genome_data(strain_id):
    
    snp_data = read_csv('../data/snp_homo/{}.homo.sum.genes.txt'.format(strain_id), sep='\t', skiprows=1)

    columns = ['Count (FRAME_SHIFT)', 'Count (START_LOST)', 'Count (STOP_GAINED)', 'Count (STOP_LOST)']
    for column in columns:
        if column not in snp_data.columns:
            snp_data[column] = 0
            
    if strain_id == 'sc23':
        deleted = set()
    else:
        cnv_data = read_csv('../data/cnv/{}.genes'.format(strain_id), sep='\t',  header=None)
        deleted = set(cnv_data[cnv_data[5] == 0][0])
    
    truncated = set(snp_data[(snp_data['Count (START_LOST)'] > 0) |
                             (snp_data['Count (FRAME_SHIFT)'] > 0) | 
                             (snp_data['Count (STOP_GAINED)'] > 0)]['#GeneId'])
    
    elongated = set(snp_data[(snp_data['Count (STOP_LOST)'] > 0)]['#GeneId'])
    
    #exclude mitochondrial genes
    deleted = [gene for gene in deleted if not gene.startswith('Q')]
    truncated = [gene for gene in truncated if not gene.startswith('Q')]
    elongated = [gene for gene in elongated if not gene.startswith('Q')]
               
    return deleted, truncated, elongated

### Get physiology data (growth and secretion rates) for a given strain

In [4]:
def get_strain_physiology(strain, biomass_rxn, glc_uptake=10):

    yield_data = read_csv('../data/product_yields_on_glucose.csv', index_col=0)
    
    mol_mass = {'Glucose': 180.1559,
             'Glycerol': 92.09382,
             'Acetate': 59.04,
             'Pyruvate': 88.06,
             'Succinate': 118.09,
             'Ethanol': 46.06844}

    products = {'glyc': 'Glycerol',
              'ac': 'Acetate',
              'pyr': 'Pyruvate',
              'succ': 'Succinate',
              'etoh': 'Ethanol' }

    met_rates = {m_id: glc_uptake * yield_data[m_name][strain] * mol_mass['Glucose'] / mol_mass[m_name]
                for m_id, m_name in products.items()}

    met_rates_sd = {m_id: glc_uptake * yield_data[m_name + '_SD'][strain] * mol_mass['Glucose'] / mol_mass[m_name]
                    for m_id, m_name in products.items()}

    growth_rate = glc_uptake * yield_data['Biomass'][strain] * mol_mass['Glucose'] / 1000.0
    growth_rate_sd = glc_uptake * yield_data['Biomass_SD'][strain] * mol_mass['Glucose'] / 1000.0

    constraints = {'R_EX_{}_e'.format(m_id): (max(met_rates[m_id] - met_rates_sd[m_id], 0),
                                              met_rates[m_id] + met_rates_sd[m_id])
                   for m_id in products}

    constraints[biomass_rxn] = (growth_rate - growth_rate_sd,
                                growth_rate + growth_rate_sd)

    constraints['R_EX_glc__D_e'] = (-glc_uptake, -glc_uptake)
    
    return constraints

### Convert gene to reaction penalty scores

In [5]:
def gene_to_reaction_penalty(model, r_id, penalties):
    gpr = model.reactions[r_id].gpr
    if not gpr:
        return 0
    else:
        return min([max([penalties.get(gene, 0) for gene in protein.genes]) 
                    for protein in gpr.proteins])

### Build a strain specific model given a set of gene penalties and minimal growth fraction

In [6]:
def strain_specific_model(model, gene_penalties, physiological_constraints):

    bigM = 1000
    
    model = model.copy()
    for r_id, (lb, ub) in physiological_constraints.items():
        model.set_flux_bounds(r_id, lb, ub)
        
    solver = solver_instance(model)
    objective = {}
    
    reaction_penalty = {}
    
    for r_id in model.reactions:
        reaction_penalty[r_id] = gene_to_reaction_penalty(model, r_id, gene_penalties)
        if reaction_penalty[r_id] > 0:
            y_i = 'y_' + r_id
            solver.add_variable(y_i, 0, 1, vartype=VarType.BINARY, update_problem=False)
            objective[y_i] = reaction_penalty[r_id]
    solver.update()
        
    for r_id, rxn in model.reactions.items():           
         if reaction_penalty[r_id] > 0:
            y_i = 'y_' + r_id
            if rxn.lb is None or rxn.lb < 0:
                solver.add_constraint('lb_' + r_id, {r_id: 1, y_i: bigM}, '>', 0, update_problem=False)
            if rxn.ub is None or rxn.ub > 0:
                solver.add_constraint('ub_' + r_id, {r_id: 1, y_i: -bigM}, '<', 0, update_problem=False)
    solver.update()
        
    solution = solver.solve(objective)

    result = [(y_id[2:], abs(w), solution.values[y_id]) for y_id, w in objective.items()]
        
    return result, solution

### Do-it-all wrapper for strain-specific model building

In [7]:
def build_costumized_model(strain_id, strain, penalties):
    model = load_original_model()

    deleted, truncated, elongated = get_genome_data(strain_id)
    gene_penalties = {}
    gene_penalties.update({gene: penalties[2] for gene in elongated})
    gene_penalties.update({gene: penalties[1] for gene in truncated})
    gene_penalties.update({gene: penalties[0] for gene in deleted})
        
    gene_penalties = {"G_" + gene.replace('-','_'): val for gene, val in gene_penalties.items()}
        
    biomass_rxn = model.detect_biomass_reaction()
    constraints = get_strain_physiology(strain, biomass_rxn)
    
    result, solution = strain_specific_model(model, gene_penalties, constraints)
    
    lost_reactions = [r_id for (r_id, _, val) in result if val == 0]
    
    #model.remove_reactions(lost_reactions)
    for r_id in lost_reactions:
        model.set_flux_bounds(r_id, 0, 0)
        
    return model, gene_penalties, lost_reactions, result, solution

### Load and initialize original model

In [8]:
def load_original_model():
    model = load_cbmodel('../data/iMM904.xml', flavor='bigg')

    glc_uptk = 10

    medium_composition = {'glc__D', 'o2', '4abz', 'btn', 'fe2', 'h', 'inost', 
                          'k', 'na1', 'nac', 'nh4', 'pi', 'pnto__R', 'so4', 'thm'}

    for m_id in medium_composition:
        model.set_lower_bound('R_EX_{}_e'.format(m_id), None)

    model.set_lower_bound('R_EX_glc__D_e', -glc_uptk)

    #model.add_gene(Gene('SPONTANEOUS'))
    #model.rules['R_H2Ot'] = '( {} or SPONTANEOUS )'.format(model.rules['R_H2Ot'])
    model.reactions['R_H2Ot'].gpr = None

    return model

### Save models to SBML

In [9]:
def save_models(models):
    for strain, model in models.items():
        path = '../models/{}.xml'.format(strain)
        save_cbmodel(model, path, flavor='bigg')

### Load and test all generated SBML files

In [10]:
def test_generated_models():
    files = glob('../models/*.xml')
    for sbmlfile in files:
        model = load_cbmodel(sbmlfile, flavor='bigg')
        strain = sbmlfile.split("/")[-1][:-4]
        sol = FBA(model)
        print strain, 'growth rate:', sol.fobj

### Helper functions for counting recovered gene frequency

In [11]:
def frequently_recovered_reactions(model, milp_results):
    count = {r_id: 0 for r_id in model.reactions}
    for res in milp_results.values():
        for r_id, w, y in res:
            if y > 0:
                count[r_id] += 1
    top = sorted(count.items(), key=lambda x: -x[1])[:10]
    for r_id, count in top:
        print count, '\t', model.rules[r_id], '\t', model.print_reaction(r_id, reaction_names=True)


def recovered_reactions_to_genes(model, lost_genes, recovered_reactions):
    recovered_genes = set()
    for r_id in recovered_reactions:
        gpr = model.reactions[r_id].gpr
        if gpr:
            recovered_genes |= set(gpr.get_genes()) & set(lost_genes)
    return recovered_genes


def frequently_recovered_genes(model, recovered_genes, penalties, gene_names, weights):
    count = {gene: [] for gene in model.genes}
    for strain, genes in recovered_genes.items():
        for gene in genes:
            count[gene].append(penalties[strain][gene])
    total = [(gene, hits) for gene, hits in count.items() if hits]
    total.sort(key=lambda (x,y) : len(y), reverse=True)
    
    frequency = []

    for gene, hits in total:
        entry = [gene, gene_names[1][gene[2:]]] + [hits.count(x) for x in weights]
        frequency.append(entry)
        
    frequency = DataFrame(frequency, columns=['gene', 'name', 'no copies', 'truncated', 'elongated'])
    
    return total, frequency

## Run pipeline

In [12]:
weights = (10, 5, 1)
strain_ids = load_strain_ids()
model = load_original_model()
models = {}
lost_genes = {}
lost_met_genes = {}
lost_reactions = {}
milp_results = {}
recovered_reactions = {}
recovered_genes = {}
solutions = {}
data = []

for strain_id, strain in strain_ids.items():
    print strain
    models[strain], lost_genes[strain], lost_reactions[strain], \
        milp_results[strain], solutions[strain] = build_costumized_model(strain_id, strain, weights)
    lost_met_genes[strain] = {gene: penalty for gene, penalty in lost_genes[strain].items() 
                              if gene in models[strain].genes}
    recovered_reactions[strain] = [x for (x,y,z) in milp_results[strain] if z > 0]
    recovered_genes[strain] = recovered_reactions_to_genes(models[strain], 
                                                           lost_met_genes[strain].keys(),
                                                           recovered_reactions[strain])
    growth = FBA(models[strain]).fobj
    data.append([strain, len(lost_genes[strain]), len(lost_met_genes[strain]), 
                 len(lost_reactions[strain]), len(recovered_reactions[strain]),
                 len(recovered_genes[strain]), growth])

summary = DataFrame(data, columns=['Strain', 'Lost Genes', 'Lost Met Genes', 
                                   'Lost Reactions', 'Recovered Reactions', 
                                   'Recovered Genes', 'Growth Rate'])


AL1
AL3-h
CA1
CBS7960
CEN.PK113-7D
CLIB215
CLIB324
CLIB382
DBVPG1373
DBVPG1788
DBVPG6044
DBVPG6765
EthanolRed
GDB135-h
GDB325
GDB379
KKYS2-h
L.1528
LU1250
NCYC110
PW5
RM11
S288c
SK1
T7
T73
UWOPS03-461.4
UWOPS05-217.3
UWOPS05-227.2
Y10
Y55
YJM269
YJM975
YJM978
YPS128
YPS606


In [13]:
summary

,Strain,Lost Genes,Lost Met Genes,Lost Reactions,Recovered Reactions,Recovered Genes,Growth Rate
0,AL1,243,13,5,1,1,0.975418
1,AL3-h,356,24,13,2,2,0.975418
2,CA1,286,23,9,3,2,0.975418
3,CBS7960,291,19,9,2,1,0.975418
4,CEN.PK113-7D,183,18,10,1,1,0.975418
5,CLIB215,316,33,19,4,3,0.975418
6,CLIB324,24,4,1,1,1,0.975418
7,CLIB382,309,22,14,0,0,0.975418
8,DBVPG1373,316,28,18,2,1,0.975418
9,DBVPG1788,468,32,24,4,3,0.975418


## Generate statistics

In [14]:
summary.to_excel('../results/deletion_results.xls', index=False)
gene_names = read_csv('../data/yeast_gene_names.csv', index_col=0, header=None)
total, frequency = frequently_recovered_genes(model, recovered_genes, lost_met_genes, gene_names, weights)
frequency.to_excel('../results/recovery_frequency.xls', index=False)

## Save models to SBML

In [15]:
save_models(models)

## Test generated models

In [16]:
test_generated_models()

AL1 growth rate: 0.975418210558
AL3-h growth rate: 0.975418210558
CA1 growth rate: 0.975418210558
CBS7960 growth rate: 0.975418210558
CEN.PK113-7D growth rate: 0.975418210558
CLIB215 growth rate: 0.975418210558
CLIB324 growth rate: 0.975418210558
CLIB382 growth rate: 0.975418210558
DBVPG1373 growth rate: 0.975418210558
DBVPG1788 growth rate: 0.975418210558
DBVPG6044 growth rate: 0.975418210558
DBVPG6765 growth rate: 0.975418210558
EthanolRed growth rate: 0.867365415806
GDB135-h growth rate: 0.975418210558
GDB325 growth rate: 0.867365415806
GDB379 growth rate: 0.975418210558
KKYS2-h growth rate: 0.867365415806
L.1528 growth rate: 0.975418210558
LU1250 growth rate: 0.975418210558
NCYC110 growth rate: 0.975418210558
PW5 growth rate: 0.975418210558
RM11 growth rate: 0.975418210558
S288c growth rate: 0.975418210558
SK1 growth rate: 0.975418210558
T7 growth rate: 0.975418210558
T73 growth rate: 0.975418210558
UWOPS03-461.4 growth rate: 0.209365937462
UWOPS05-217.3 growth rate: 0.975418210558